In [1]:
model_path = "t2wml_project/"
data_file = "{}/Ethiopia_Daily_Data_Multiple_Stations.xlsx".format(model_path)
model_file = "{}/discharge_model.yaml".format(model_path)
wikifier_file = "{}/wikify_region_output.csv".format(model_path)
sheet_mapping = {
    "Baro": "Baro-Masha",
    "Bulbul": "Bulbul-Serbo",
    "Dincha": "Dincha-Bonga",
    "Gecha": "Gecha-Bonga",
    "Asendabo": "Gilgelghibe-Asendabo",
    "Gojeb": "Gojeb-Shebe",
    "Ouwa": "Ouwa-Gulliso",
    "Sheta": "Sheta-Bonga",
    "Sori": "Sore-Metu",
    "Woshi": "Woshi-Dimbira"
}
output_path = "data/"
dataset_original = "{}/temp_kgtk.tsv".format(output_path)
dataset_id = "{}/temp_kgtk_add_id.tsv".format(output_path)
dataset_implode = "{}/temp_kgtk_implode.tsv".format(output_path)
results_path = "{}/results".format(output_path)
output_file = "{}/{}_Discharge_2015-10_to_2018-10.csv"

In [2]:
import pandas as pd
from t2wml.knowledge_graph import KnowledgeGraph

In [3]:
def generate_kgtk_file(data, sheet, model, wikifier, output_path):
    kg = KnowledgeGraph.generate_from_files(data, sheet, model, wikifier)
    kg.save_file(output_path, "kgtk")

In [ ]:
for sheet_name, result_name in sheet_mapping.items(): 
    result_file_path = output_file.format(results_path, result_name)
    generate_kgtk_file(data_file, sheet_name, model_file, wikifier_file, dataset_original)
    
    !kgtk add-id -i $dataset_original -o $dataset_id --id-style node1-label-num
    !kgtk implode -i $dataset_id -o $dataset_implode --without language_suffix si_units units_node 2>/dev/null
    !kgtk query -i $dataset_implode \
    --match '(n1)-[r]->(n_discharge), \
        (r)-[r_date:P585]->(n_date), \
        (r)-[r_area:P2046]->(n_area), \
        (r)-[r_location:P625]->(n_location)' \
    --return 'distinct kgtk_date_date(n_date) as date, \
        n_discharge as `discharge(cumecs)`, \
        n_area as `area(sq km)`, \
        n_location.`kgtk:latitude` as latitude, n_location.`kgtk:longitude` as longitude' \
    -o $result_file_path

    csv_table=pd.read_table(result_file_path,sep='\t')
    csv_table['date'] = csv_table['date'].str[1:] # Remove caret symbol from dates
    csv_table.to_csv(result_file_path,index=False)